In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

## All Lxy Bin

In [4]:
prefix = './coffea/skimmed/'
suffix = '_ctau-100-BDT-based_AllLxy'

In [5]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [6]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-100-BDT-based_Background_Efficiency_ctau-100_AllLxyBin'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-100-BDT-based_Background_Efficiency_ctau-100_AllLxyBin.csv


In [8]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,0.07
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,0.58
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",9.31
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",2.68
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",12.65


In [9]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-100-BDT-based_Background_EventCount_Weighted_ctau-100_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-100-BDT-based_Background_EventCount_Weighted_ctau-100_AllLxyBin.csv


In [10]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,0
Top,49818,4688,3058,2348,2337,2336,2006,2
DY,205,13,11,11,11,11,9,0
Diboson,18067,1234,915,860,852,848,767,2
Triboson,2257,170,116,106,105,104,92,0
W+jets,697657,48425,37015,35472,35415,35377,29090,34
Z+jets,1281459,67427,52627,50986,50934,50903,44480,21
Total,2061774,122338,93902,89942,89813,89737,76447,59


In [11]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-100-BDT-based_Background_EventCount_Raw_ctau-100_AllLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-100-BDT-based_Background_EventCount_Raw_ctau-100_AllLxyBin.csv


In [12]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 12.65


In [13]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [14]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-100-BDT-based_AllLxy.coffea')[0]

In [15]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.2,0.5,100.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-100
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,52.5,5.0,100.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-100
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,44.0,8.0,100.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-100
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,42.0,4.0,100.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-100
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,33.0,6.0,100.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-100
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,31.5,3.0,100.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-100
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,22.0,4.0,100.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-100
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,11.0,2.0,100.0,10.0,12.0,0.2,sig_2018_Mchi-11p0_dMchi-2p0_ctau-100


In [16]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT v2 Loose WP']

### Efficiency

In [17]:
pd.options.display.float_format = '{:,.6f}'.format

In [18]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [19]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [20]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000054
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.000536
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000038
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.002231,0.001905,0.001847,0.001847,0.001847,0.001804,0.000441
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.001802,0.001476,0.001411,0.001411,0.001411,0.001364,0.000371
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.001657,0.001419,0.001371,0.001371,0.001371,0.001331,0.000315
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.001285,0.001082,0.001052,0.001052,0.001052,0.001011,0.000251
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.001197,0.001041,0.000995,0.000995,0.000995,0.000966,0.000185
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.000922,0.000793,0.000760,0.000760,0.000760,0.000722,0.000167
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.007977,0.000601,0.000507,0.000483,0.000483,0.000483,0.000467,0.000075


In [21]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_ctau-100-BDT-based_Signal_ctau-100_samples_Efficiency_ctau-100_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_ctau-100-BDT-based_Signal_ctau-100_samples_Efficiency_ctau-100_AllLxyBin.csv


### Weighted Event count

In [22]:
pd.options.display.float_format = '{:,.2f}'.format

In [23]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [24]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [25]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,2.33
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.86
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.07
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,1.94
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,"1,302.77",123.25,103.80,100.85,100.85,100.85,96.92,24.11
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,"24,707.11","2,184.60","1,899.41","1,816.96","1,816.96","1,816.96","1,763.83",338.37
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.10
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.07


In [26]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_ctau-100-BDT-based_Signal_ctau-100_samples_EventCount_Weighted_ctau-100_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_ctau-100-BDT-based_Signal_ctau-100_samples_EventCount_Weighted_ctau-100_AllLxyBin.csv


In [27]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,2.33
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.86
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.07
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,1.94
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.10
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.07
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,2.23
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,238.69,19.21,16.19,15.29,15.29,15.29,14.98,2.05


### Raw count

In [28]:
pd.options.display.float_format = '{:,.0f}'.format

In [29]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [30]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [31]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,165,155,155,155,147,16
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,643,617,617,617,590,167
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,735,692,692,692,666,56
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,15529,1539,1314,1274,1274,1274,1244,304
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,10574,1084,888,849,849,849,821,223
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,16226,1522,1303,1259,1259,1259,1222,289
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,5296,501,422,410,410,410,394,98
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,14389,1272,1106,1058,1058,1058,1027,197
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6241,557,479,459,459,459,436,101
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,5842,440,371,354,354,354,342,55


In [32]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_ctau-100-BDT-based_Signal_ctau-100_samples_EventCount_Raw_ctau-100_AllLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_ctau-100-BDT-based_Signal_ctau-100_samples_EventCount_Raw_ctau-100_AllLxyBin.csv


In [33]:
nBkg

12.654182014616387

In [34]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [35]:
pd.options.display.float_format = '{:,.4f}'.format

In [36]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [37]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.1050,5.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,2.3320,5.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0716,10.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,2.0463,10.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.0994,20.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,2.2315,20.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,24.1107,30.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,338.3730,30.0000,0.1000,100.0000,12.6542
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.0707,40.0000,0.2000,100.0000,12.6542
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,1.9383,40.0000,0.1000,100.0000,12.6542


In [38]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.1050,12.6542,0.0295
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,2.3320,12.6542,0.6556
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0716,12.6542,0.0201
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,2.0463,12.6542,0.5752
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0994,12.6542,0.0279
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,2.2315,12.6542,0.6273
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,24.1107,12.6542,6.7779
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,338.3730,12.6542,95.1215
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0707,12.6542,0.0199
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,1.9383,12.6542,0.5449


In [39]:
outdir = './csv/SR_Yields/'
outname = 'SR_ctau-100-BDT-based_ctau-100_AllLxy_S_over_B.csv'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_ctau-100-BDT-based_ctau-100_AllLxy_S_over_B.csv.csv


### ctau = 100mm signal points

In [40]:
# S/sqrt(B) in ctau = 10mm, delta = 0.1
ct = 100.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,2.3320,12.6542,0.6556
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,2.0463,12.6542,0.5752
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,2.2315,12.6542,0.6273
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,338.3730,12.6542,95.1215
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,1.9383,12.6542,0.5449
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0000,0.1000,100.0000,0.8567,12.6542,0.2408


In [41]:
# S/sqrt(B) in ctau = 10mm, delta = 0.2
ct = 100.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.1050,12.6542,0.0295
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0716,12.6542,0.0201
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0994,12.6542,0.0279
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,24.1107,12.6542,6.7779
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0707,12.6542,0.0199
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0000,0.2000,100.0000,0.0327,12.6542,0.0092


# SR BDT-ctau-100: High Lxy (Lxy > 0.5)

## Background

In [42]:
prefix = './coffea/skimmed/'
suffix = '_ctau-100-BDT-based_HighLxy'

In [43]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [44]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000003,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000015,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000034,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000066,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000173,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [45]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,7.84,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,139.54,0.07
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,48.18,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,103.12,0.29
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,0.93,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","6,454.21",7.89
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","3,694.44",2.45
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","10,448.27",10.70


In [46]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,1,0
Top,49818,4688,3058,2348,2337,2336,2006,972,2
DY,205,13,11,11,11,11,9,2,0
Diboson,18067,1234,915,860,852,848,767,419,1
Triboson,2257,170,116,106,105,104,92,50,0
W+jets,697657,48425,37015,35472,35415,35377,29090,16594,30
Z+jets,1281459,67427,52627,50986,50934,50903,44480,26108,18
Total,2061774,122338,93902,89942,89813,89737,76447,44146,51


In [47]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 10.70


## Signal

In [48]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [49]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-100-BDT-based_HighLxy.coffea')[0]

In [50]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.2,0.5,100.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-100
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,52.5,5.0,100.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-100
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,42.0,4.0,100.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-100
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,33.0,6.0,100.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-100
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,44.0,8.0,100.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-100
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,31.5,3.0,100.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-100
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,11.0,2.0,100.0,10.0,12.0,0.2,sig_2018_Mchi-11p0_dMchi-2p0_ctau-100
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,22.0,4.0,100.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-100


In [51]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy > 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [52]:
pd.options.display.float_format = '{:,.6f}'.format

In [53]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [54]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [55]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.001409,0.000472
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000351,0.000051
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000276,0.000032
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.002231,0.001905,0.001847,0.001847,0.001847,0.001804,0.001340,0.000395
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.001657,0.001419,0.001371,0.001371,0.001371,0.001331,0.000968,0.000291
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.001285,0.001082,0.001052,0.001052,0.001052,0.001011,0.000772,0.000226
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.001802,0.001476,0.001411,0.001411,0.001411,0.001364,0.001045,0.000337
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.001197,0.001041,0.000995,0.000995,0.000995,0.000966,0.000682,0.000176
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.007977,0.000601,0.000507,0.000483,0.000483,0.000483,0.000467,0.000329,0.000074
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.000922,0.000793,0.000760,0.000760,0.000760,0.000722,0.000517,0.000147


### Weighted Event count

In [56]:
pd.options.display.float_format = '{:,.2f}'.format

In [57]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [58]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [59]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.09,0.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.68,0.10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,16.78,1.92
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,2.60,0.77
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,5.96,1.79
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,"1,302.77",123.25,103.80,100.85,100.85,100.85,96.92,74.06,21.65
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.20,0.06
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,"24,707.11","2,184.60","1,899.41","1,816.96","1,816.96","1,816.96","1,763.83","1,245.14",321.17
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.31,0.07
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.31,0.09


In [61]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.09,0.03
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.68,0.10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,16.78,1.92
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,2.60,0.77
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,5.96,1.79
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.20,0.06
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.31,0.07
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.31,0.09
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,9.63,2.16
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,238.69,19.21,16.19,15.29,15.29,15.29,14.98,10.02,1.91


### Raw count

In [62]:
pd.options.display.float_format = '{:,.0f}'.format

In [63]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [64]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [65]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy > 0.5,BDT v2 Loose WP
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,643,617,617,617,590,439,147
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,165,155,155,155,147,103,15
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,735,692,692,692,666,403,46
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,15529,1539,1314,1274,1274,1274,1244,924,272
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,16226,1522,1303,1259,1259,1259,1222,889,267
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,5296,501,422,410,410,410,394,301,88
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,10574,1084,888,849,849,849,821,629,203
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,14389,1272,1106,1058,1058,1058,1027,725,187
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,5842,440,371,354,354,354,342,241,54
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6241,557,479,459,459,459,436,312,89


In [66]:
nBkg

10.700309013462268

In [67]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [68]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [69]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.0984,5.0000,0.2000,100.0000,10.7003
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,1.9156,5.0000,0.1000,100.0000,10.7003
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0703,10.0000,0.2000,100.0000,10.7003
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,1.9114,10.0000,0.1000,100.0000,10.7003
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.0876,20.0000,0.2000,100.0000,10.7003
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,2.1577,20.0000,0.1000,100.0000,10.7003
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,21.6507,30.0000,0.2000,100.0000,10.7003
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,321.1708,30.0000,0.1000,100.0000,10.7003
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,1.7909,40.0000,0.1000,100.0000,10.7003
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.0643,40.0000,0.2000,100.0000,10.7003


In [70]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0984,10.7003,0.0301
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,1.9156,10.7003,0.5856
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0703,10.7003,0.0215
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,1.9114,10.7003,0.5843
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0876,10.7003,0.0268
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,2.1577,10.7003,0.6596
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,21.6507,10.7003,6.6187
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,321.1708,10.7003,98.1833
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,1.7909,10.7003,0.5475
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0643,10.7003,0.0197


# SR BDT-ctau-100: Low Lxy (Lxy <= 0.5)

## Background

In [71]:
prefix = './coffea/skimmed/'
suffix = '_ctau-100-BDT-based_LowLxy'

In [72]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [73]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000004,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000012,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000042,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000048,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000120,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [74]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,624.41,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,154.46,0.00
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,407.47,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,86.23,0.29
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,1.16,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","4,750.18",1.43
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","2,562.32",0.24
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","8,586.23",1.95


In [75]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
QCD,12311,381,160,159,159,158,3,2,0
Top,49818,4688,3058,2348,2337,2336,2006,1034,0
DY,205,13,11,11,11,11,9,7,0
Diboson,18067,1234,915,860,852,848,767,348,1
Triboson,2257,170,116,106,105,104,92,42,0
W+jets,697657,48425,37015,35472,35415,35377,29090,12496,4
Z+jets,1281459,67427,52627,50986,50934,50903,44480,18372,3
Total,2061774,122338,93902,89942,89813,89737,76447,32301,8


In [76]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 1.95


## Signal

In [77]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [78]:
# Signal
sig_histo = util.load(f'{prefix}/ctau-100-BDT-based_LowLxy.coffea')[0]

In [79]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.2,0.5,100.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-100
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,52.5,5.0,100.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-100
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,44.0,8.0,100.0,40.0,48.0,0.2,sig_2018_Mchi-44p0_dMchi-8p0_ctau-100
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,33.0,6.0,100.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-100
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,42.0,4.0,100.0,40.0,44.0,0.1,sig_2018_Mchi-42p0_dMchi-4p0_ctau-100
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,31.5,3.0,100.0,30.0,33.0,0.1,sig_2018_Mchi-31p5_dMchi-3p0_ctau-100
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,21.0,2.0,100.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-100
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,22.0,4.0,100.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-100


In [80]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy <= 0.5',
 'BDT v2 Loose WP']

### Efficiency

In [81]:
pd.options.display.float_format = '{:,.6f}'.format

In [82]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [83]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [84]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000150,0.000003
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.000485,0.000064
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000180,0.000007
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.002231,0.001905,0.001847,0.001847,0.001847,0.001804,0.000464,0.000046
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.001802,0.001476,0.001411,0.001411,0.001411,0.001364,0.000319,0.000033
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.001285,0.001082,0.001052,0.001052,0.001052,0.001011,0.000238,0.000026
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.001657,0.001419,0.001371,0.001371,0.001371,0.001331,0.000363,0.000024
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.001197,0.001041,0.000995,0.000995,0.000995,0.000966,0.000284,0.000009
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,0.010649,0.000928,0.000818,0.000775,0.000775,0.000775,0.000749,0.000238,0.000004
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.000922,0.000793,0.000760,0.000760,0.000760,0.000722,0.000205,0.000020


### Weighted Event count

In [85]:
pd.options.display.float_format = '{:,.2f}'.format

In [86]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [87]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [88]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.29,0.01
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03,0.00
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,10.95,0.42
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.90,0.09
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.06,0.01
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,"1,302.77",123.25,103.80,100.85,100.85,100.85,96.92,22.86,2.46
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.23,0.15
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,"24,707.11","2,184.60","1,899.41","1,816.96","1,816.96","1,816.96","1,763.83",518.69,17.20
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.49,0.07
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.12,0.01


In [89]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.29,0.01
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03,0.00
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,10.95,0.42
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.90,0.09
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.06,0.01
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.23,0.15
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.49,0.07
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.12,0.01
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.13,0.00
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,238.69,19.21,16.19,15.29,15.29,15.29,14.98,4.96,0.13


### Raw count

In [90]:
pd.options.display.float_format = '{:,.0f}'.format

In [91]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [92]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [93]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT v2 Loose WP
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,165,155,155,155,147,44,1
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,643,617,617,617,590,151,20
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,735,692,692,692,666,263,10
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,15529,1539,1314,1274,1274,1274,1244,320,32
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,10574,1084,888,849,849,849,821,192,20
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,5296,501,422,410,410,410,394,93,10
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,16226,1522,1303,1259,1259,1259,1222,333,22
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,14389,1272,1106,1058,1058,1058,1027,302,10
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,8195,714,629,596,596,596,576,183,3
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6241,557,479,459,459,459,436,124,12


In [94]:
nBkg

1.953873001154117

In [95]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [96]:
pd.options.display.float_format = '{:,.4f}'.format

df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [97]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.0066,5.0000,0.2000,100.0000,1.9539
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.4165,5.0000,0.1000,100.0000,1.9539
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0013,10.0000,0.2000,100.0000,1.9539
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,0.1349,10.0000,0.1000,100.0000,1.9539
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,0.0738,20.0000,0.1000,100.0000,1.9539
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.0118,20.0000,0.2000,100.0000,1.9539
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,2.4600,30.0000,0.2000,100.0000,1.9539
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,17.2022,30.0000,0.1000,100.0000,1.9539
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.0063,40.0000,0.2000,100.0000,1.9539
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.1475,40.0000,0.1000,100.0000,1.9539


In [98]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0066,1.9539,0.0047
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,0.4165,1.9539,0.2979
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0013,1.9539,0.0009
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,0.1349,1.9539,0.0965
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,0.0738,1.9539,0.0528
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0118,1.9539,0.0084
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,2.4600,1.9539,1.7599
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,17.2022,1.9539,12.3065
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0063,1.9539,0.0045
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,0.1475,1.9539,0.1055
